# ClioMakeUp Survey Data Analysis

## Dataframe preparation

### Read Excel response file and do initial cleanup

In [1]:
import pandas as pd
from IPython.display import display

In [2]:
# Read response file
response_file = '../data/risposte.xlsx'
df = pd.read_excel(response_file)

# Remove timestamps
df = df.drop(df.columns[0], axis=1)
df.head()

,Hai mai acquistato prodotti di ClioMakeUp?,Con che frequenza acquisti da ClioMakeUp? (Sia nell'e-commerce che negli store),Dove effettui gli acquisti?,Da quanto tempo conosci la linea di ClioMakeUp?,Quali prodotti di ClioMakeUp hai comprato durante il tuo ultimo acquisto?,"Quali prodotti hai comprato del brand ClioMakeUp? (Per favore includi tutti i prodotti acquistati, anche quelli del tuo ultimo acquisto)",Sei rimasto soddisfatto dopo l'utilizzo dei prodotti di ClioMakeUp?,Genere,Età (In anni compiuti),Titolo di studio conseguito,Quali delle seguenti categorie evidenziano al meglio la tua attuale condizione?,Dove vivi?,In quale provincia vivi?,Per chi acquisti i prodotti di ClioMakeUp?
0,Si,da 1 a 2 volte l'anno,"E-commerce, PopUp Store",5 anni (dal lancio dei primi rossetti),Rossetti liquidi LiquidLove,"Mascara DarkLove, Eyeliner DeepLove, Rossetti ...","Sì, ma non di tutti",Femmina,38,Laurea triennale,Lavoratore dipendente a tempo indeterminato,Italia,Alessandria,"Me, Famiglia, Amic*"
1,No,NaN,NaN,NaN,NaN,NaN,NaN,Maschio,19,Scuola secondaria di secondo grado,Studente,Italia,Alessandria,NaN
2,No,NaN,NaN,NaN,NaN,NaN,NaN,Femmina,26,Master,Lavoratore dipendente a tempo indeterminato,Italia,Alessandria,NaN
3,Si,da 1 a 2 volte l'anno,ClioMakeUp Experience Store,5 anni (dal lancio dei primi rossetti),Rossetti liquidi LiquidLove,"Fondotinta OhMyLove, Correttore OhMyLove, Pale...",Sì,Femmina,38,Scuola secondaria di secondo grado,Disoccupata\o,Italia,Alessandria,Me
4,Si,da 5 a 6 volte l'anno,E-commerce,5 anni (dal lancio dei primi rossetti),"Correttore OhMyLove, Ombretti cremosi SweetieL...","Correttore OhMyLove, Blush CuteLove, Illuminan...",Assolutamente si,Femmina,25,Laurea magistrale,Lavoratore dipendente a tempo indeterminato,Italia,Ancona,Me


In [3]:
# Rename headers
headers = ['Customer',
           'Frequency',
           'PurchaseLocation',
           'BrandKnownFor',
           'LastPurchase',
           'AllPurchases',
           'Satisfaction',
           'Gender',
           'Age',
           'Education',
           'Profession',
           'PlaceOfResidence',
           'Province',
           'SelfOrOthers'
]
df.columns = headers
df.head()

,Customer,Frequency,PurchaseLocation,BrandKnownFor,LastPurchase,AllPurchases,Satisfaction,Gender,Age,Education,Profession,PlaceOfResidence,Province,SelfOrOthers
0,Si,da 1 a 2 volte l'anno,"E-commerce, PopUp Store",5 anni (dal lancio dei primi rossetti),Rossetti liquidi LiquidLove,"Mascara DarkLove, Eyeliner DeepLove, Rossetti ...","Sì, ma non di tutti",Femmina,38,Laurea triennale,Lavoratore dipendente a tempo indeterminato,Italia,Alessandria,"Me, Famiglia, Amic*"
1,No,NaN,NaN,NaN,NaN,NaN,NaN,Maschio,19,Scuola secondaria di secondo grado,Studente,Italia,Alessandria,NaN
2,No,NaN,NaN,NaN,NaN,NaN,NaN,Femmina,26,Master,Lavoratore dipendente a tempo indeterminato,Italia,Alessandria,NaN
3,Si,da 1 a 2 volte l'anno,ClioMakeUp Experience Store,5 anni (dal lancio dei primi rossetti),Rossetti liquidi LiquidLove,"Fondotinta OhMyLove, Correttore OhMyLove, Pale...",Sì,Femmina,38,Scuola secondaria di secondo grado,Disoccupata\o,Italia,Alessandria,Me
4,Si,da 5 a 6 volte l'anno,E-commerce,5 anni (dal lancio dei primi rossetti),"Correttore OhMyLove, Ombretti cremosi SweetieL...","Correttore OhMyLove, Blush CuteLove, Illuminan...",Assolutamente si,Femmina,25,Laurea magistrale,Lavoratore dipendente a tempo indeterminato,Italia,Ancona,Me


### Clean up column by column

In [4]:
# Function to:
# - set 'category' data type for the column
# - define the complete list of categories (even those that never appear in the data) and their order
# - rename the categories from Italian to English
def fill_rename_categories(s, categories):
    # Set categorical dtype
    s = s.astype('category')

    # Set categories, define unused and define order
    s = s.cat.set_categories(list(categories.keys()), ordered=True)

    # Translate to English
    s = s.cat.rename_categories(categories)

    return s

# Function to check all the answers reported in the data for a column (a question) againsts a list of valid answers
def check_spurious_values(s, value_list, multiple=False):
    # If an entry can contain multiple values, split them
    if multiple:
        s = s.str.split(',').map(lambda item: [i.strip() for i in item], na_action='ignore')
        s = s.explode(column)
    
    # Compare values in the column with valid values
    vc = s.value_counts()
    column_values = vc.index
    valid = ['Ok' if value in value_list else 'Invalid' for value in column_values]

    display(pd.DataFrame({'Occurrences': vc, 'Valid': valid}))
    

#### Customer

In [5]:
column = 'Customer'
categories = {
    'Si': 'Yes',
    'No': 'No',
}
df[column] = fill_rename_categories(df[column], categories)

#### Frequency

In [6]:
column = 'Frequency'
categories = {
    "da 1 a 2 volte l'anno": "1-2 times a year",
    "da 3 a 4 volte l'anno": "3-4 times a year",
    "da 5 a 6 volte l'anno": "5-6 times a year",
    "da 7 a 8 volte l'anno": "7-8 times a year",
    "più di 9 volte l'anno": "> 9 times a year",
}
df[column] = fill_rename_categories(df[column], categories)

#### PurchaseLocation
The question contains the 'Other' option, so we need to check and correct spurious values.

In [7]:
column = 'PurchaseLocation'
categories = ['E-commerce', 'PopUp Store', 'ClioMakeUp Experience Store']

# Check valid answers
check_spurious_values(df[column], categories, multiple=True)

,Occurrences,Valid
E-commerce,104,Ok
PopUp Store,48,Ok
ClioMakeUp Experience Store,12,Ok
On-line,1,Invalid


In [8]:
# Clean data (change spurious 'On-line' to 'E-commerce')
df[column] = df[column].str.replace('On-line', 'E-commerce')
check_spurious_values(df[column], categories, multiple=True)

,Occurrences,Valid
E-commerce,105,Ok
PopUp Store,48,Ok
ClioMakeUp Experience Store,12,Ok


#### BrandKnownFor

In [9]:
column = 'BrandKnownFor'
categories = {
    'Meno di un anno':                          'less than 1 year',
    '1 anno':                                   '1 year',
    '2 anni':                                   '2 years',
    '3 anni':                                   '3 years',
    '4 anni':                                   '4 years',
    '5 anni (dal lancio dei primi rossetti)':   '5 years (first lipstick launch)',
}

# Check valid answers
check_spurious_values(df[column], categories)

,Occurrences,Valid
5 anni (dal lancio dei primi rossetti),96,Ok
2 anni,12,Ok
4 anni,9,Ok
3 anni,8,Ok
1 anno,4,Ok
Meno di un anno,3,Ok
Da molto più di 5 anni,1,Invalid


In [10]:
# Clean data
df[column] = df[column].str.replace('Da molto più di 5 anni', '5 anni (dal lancio dei primi rossetti)')
check_spurious_values(df[column], categories)

,Occurrences,Valid
5 anni (dal lancio dei primi rossetti),97,Ok
2 anni,12,Ok
4 anni,9,Ok
3 anni,8,Ok
1 anno,4,Ok
Meno di un anno,3,Ok


In [11]:
# Set categories
df[column] = fill_rename_categories(df[column], categories)

#### LastPurchase
This data is used to derive the average receipt amount for each customer

In [12]:
column = 'LastPurchase'
# Define price dictionary
prices_dict = {
    'WonderMousse (Mousse viso detergente e struccante)':   15.50,
    'SuperStrucco (Balsamo detergente e struccante)':       24.50,
    'Kit doppia detersione (WonderMousse + SuperStrucco)':  35.00,
    'Paciocchi (Maschera occhi)':                           6.50,
    'Kit Paciocchi 5+1':                                    32.50,
    'Fondotinta OhMyLove':                                  28.50,
    'Correttore OhMyLove':                                  17.50,
    'Blush CuteLove':                                       15.50,
    'Illuminante CosmicLove':                               16.50,
    'Matite Sopracciglia AllDayLov Brow':                   12.50,
    'Ombretti liquidi SweetieLove':                         14.50,
    'Ombretti cremosi SweetieLove':                         14.50,
    'Mascara DarkLove':                                     14.50,
    'Eyeliner DeepLove':                                    14.50,
    'Matita occhi AllDayLove':                              10.50,
    'Palette BeautyLove':                                   57.50,
    'Palette FirstLove':                                    32.50,
    'Palette MyFirstLove ( 4 ombretti)':                    5.50,
    'Ombretti ClioMakeUp Eyeshadow':                        6.50,
    'Ultrabalm Passion (Limited Edition)':                  15.50,
    'Ultrabalm CoccoLove':                                  15.50,
    'Lip balm colorati CoccoLove':                          12.50,
    'Rossetti liquidi LiquidLove':                          13.50,
    'Rossetti cremosi CreamyLove':                          12.50,
    'Pennelli':                                             22.00,
}
# Create new column with receipt amount
new_column = 'ReceiptAmount'
df[new_column] = df[column].map(lambda item: sum([prices_dict[i.strip()] for i in item.split(',') if i.strip() in prices_dict]), na_action='ignore')

#### AllPurchases

In [13]:
column = 'AllPurchases'
categories = list(prices_dict.keys())

# Check valid answers
check_spurious_values(df[column], categories, multiple=True)

,Occurrences,Valid
Rossetti liquidi LiquidLove,76,Ok
Rossetti cremosi CreamyLove,67,Ok
Mascara DarkLove,62,Ok
Correttore OhMyLove,48,Ok
Ombretti cremosi SweetieLove,42,Ok
Fondotinta OhMyLove,31,Ok
Lip balm colorati CoccoLove,31,Ok
Eyeliner DeepLove,29,Ok
Matita occhi AllDayLove,29,Ok
Blush CuteLove,22,Ok


In [14]:
# Clean data
df[column] = df[column].str.replace('passion', 'Passion')
check_spurious_values(df[column], categories, multiple=True)
# Turn single strings into list of strings
df[column] = df[column].str.split(',').map(lambda item: [i.strip() for i in item], na_action='ignore')

,Occurrences,Valid
Rossetti liquidi LiquidLove,76,Ok
Rossetti cremosi CreamyLove,67,Ok
Mascara DarkLove,62,Ok
Correttore OhMyLove,48,Ok
Ombretti cremosi SweetieLove,42,Ok
Fondotinta OhMyLove,31,Ok
Lip balm colorati CoccoLove,31,Ok
Eyeliner DeepLove,29,Ok
Matita occhi AllDayLove,29,Ok
Blush CuteLove,22,Ok


#### Satisfaction

In [15]:
column = 'Satisfaction'
categories = {
    "Assolutamente si":     "Definitely yes",
    "Sì":                   "Yes",
    "Sì, ma non di tutti":  "Yes, but not all",
    "No":                   "No",
    "Assolutamente no":     "Definitely no",
}

# Check valid answers
check_spurious_values(df[column], categories)

,Occurrences,Valid
Assolutamente si,56,Ok
Sì,44,Ok
"Sì, ma non di tutti",28,Ok
No,3,Ok
Assolutamente no,2,Ok


No invalid answers, so we just set the categories. Nothing to correct.

In [16]:
# Set categories
df[column] = fill_rename_categories(df[column], categories)

#### Gender

In [17]:
column = 'Gender'
categories = {
    "Maschio":  "Male",
    "Femmina":  "Female",
    "Altro":    "Other",
}
# Set categories
df[column] = fill_rename_categories(df[column], categories)

#### Age

In [18]:
column = 'Age'
# Just set the data type as integer
df[column] = df[column].astype('int')

#### Education

In [19]:
column = 'Education'
categories = {
    "Licenza media":                        "Middle school",
    "Scuola secondaria di secondo grado":   "High school",
    "Laurea triennale":                     "Bachelor's degree",
    "Laurea magistrale":                    "Master's degree",
    "Master":                               "Postgraduate specialisation",
    "Dottorato di ricerca":                 "PhD",
}

# Check valid answers
check_spurious_values(df[column], categories)

,Occurrences,Valid
Scuola secondaria di secondo grado,114,Ok
Laurea triennale,107,Ok
Laurea magistrale,71,Ok
Master,20,Ok
Licenza media,11,Ok
Dottorato di ricerca,4,Ok
Acconciatura,1,Invalid
"Estetista, onicotenica",1,Invalid


We make some assumptions on the education for the two invalid answers so we can still use those entries.

In [20]:
# Clean data
df[column] = df[column].str.replace('Acconciatura', 'Licenza media')
index = df[df[column].str.contains('Estetista, onicotenica')].index[0]
df.at[index, column] = 'Scuola secondaria di secondo grado'
check_spurious_values(df[column], categories)

,Occurrences,Valid
Scuola secondaria di secondo grado,115,Ok
Laurea triennale,107,Ok
Laurea magistrale,71,Ok
Master,20,Ok
Licenza media,12,Ok
Dottorato di ricerca,4,Ok


In [21]:
# Set categories
df[column] = fill_rename_categories(df[column], categories)

#### Profession

In [22]:
column = 'Profession'
categories = {
    "Studente":                                     "Student",
    "Stagista":                                     "Intern",
    "Lavoratore autonomo":                          "Self-employed",
    "Lavoratore dipendente a tempo determinato":    "Employed (fixed-term contract)",
    "Lavoratore dipendente a tempo indeterminato":  "Employed (permanent contract)",
    "Disoccupata\o":                                "Unemployed",
    "Altro":                                        "Other",
}
df[column] = fill_rename_categories(df[column], categories)

#### SelfOrOthers

In [23]:
column = 'SelfOrOthers'
categories = {
    "Me":       "Me",
    "Famiglia": "Family",
    "Amic*":    "Friends",
    "Partner":  "Partner",
}

# Check valid answers
check_spurious_values(df[column], categories, multiple=True)

,Occurrences,Valid
Me,129,Ok
Famiglia,29,Ok
Amic*,17,Ok
Per il mio kit professionale,1,Invalid
Partner,1,Ok


In [24]:
# Clean data
df[column] = df[column].str.replace('Per il mio kit professionale', 'Me')
check_spurious_values(df[column], categories, multiple=True)

,Occurrences,Valid
Me,130,Ok
Famiglia,29,Ok
Amic*,17,Ok
Partner,1,Ok


In [25]:
# Translate data
# We can't use the fill_rename_categories() method because each entry is a single string of multiple answers
for key, value in categories.items():
    df[column] = df[column].str.replace(key, value, regex=False)

In [26]:
# Turn single strings into list of strings
df[column] = df[column].str.split(',').map(lambda item: [i.strip() for i in item], na_action='ignore')

Show the final resulting dataframe

In [27]:
df.head()

,Customer,Frequency,PurchaseLocation,BrandKnownFor,LastPurchase,AllPurchases,Satisfaction,Gender,Age,Education,Profession,PlaceOfResidence,Province,SelfOrOthers,ReceiptAmount
0,Yes,1-2 times a year,"E-commerce, PopUp Store",5 years (first lipstick launch),Rossetti liquidi LiquidLove,"[Mascara DarkLove, Eyeliner DeepLove, Rossetti...","Yes, but not all",Female,38,Bachelor's degree,Employed (permanent contract),Italia,Alessandria,"[Me, Family, Friends]",13.5
1,No,NaN,NaN,NaN,NaN,NaN,NaN,Male,19,High school,Student,Italia,Alessandria,NaN,NaN
2,No,NaN,NaN,NaN,NaN,NaN,NaN,Female,26,Postgraduate specialisation,Employed (permanent contract),Italia,Alessandria,NaN,NaN
3,Yes,1-2 times a year,ClioMakeUp Experience Store,5 years (first lipstick launch),Rossetti liquidi LiquidLove,"[Fondotinta OhMyLove, Correttore OhMyLove, Pal...",Yes,Female,38,High school,Unemployed,Italia,Alessandria,[Me],13.5
4,Yes,5-6 times a year,E-commerce,5 years (first lipstick launch),"Correttore OhMyLove, Ombretti cremosi SweetieL...","[Correttore OhMyLove, Blush CuteLove, Illumina...",Definitely yes,Female,25,Master's degree,Employed (permanent contract),Italia,Ancona,[Me],45.5


## Demographic analysis

In [28]:
# Imports matplotlib and helper stuff for plots
import plot_config as pcfg

### Gender

In [29]:
df['Gender'].value_counts().plot.pie(**pcfg.pie_kwargs)
pcfg.save_fig('dem01_gender')

### Age

In [30]:
# See info about age
df['Age'].describe()

count    329.000000
mean      28.085106
std        8.193472
min       18.000000
25%       23.000000
50%       26.000000
75%       31.000000
max       66.000000
Name: Age, dtype: float64

In [31]:
# Create age groups
df['AgeGroup'] = pd.cut(df['Age'], [0, 25, 35, 45, 1000], labels=['<25', '25-35', '35-45', '>45'])
df['AgeGroup'].value_counts().sort_index()

<25      155
25-35    127
35-45     30
>45       17
Name: AgeGroup, dtype: int64

In [32]:
ax = df['AgeGroup'].value_counts().sort_index().plot.bar(**pcfg.bar_kwargs, color=pcfg.random_color())
pcfg.hide_spines(ax)
pcfg.save_fig('dem02_age')

### Education

In [33]:
df['Education'].value_counts().sort_index()

Middle school                   12
High school                    115
Bachelor's degree              107
Master's degree                 71
Postgraduate specialisation     20
PhD                              4
Name: Education, dtype: int64

In [34]:
x = df['Education'].value_counts(normalize=True).sort_index(ascending=False) * 100
ax = x.plot.barh(**pcfg.bar_kwargs, color=pcfg.random_color())
ax.set_xlabel('%')
pcfg.hide_spines(ax)
pcfg.save_fig('dem03_education')

### Profession

In [35]:
df['Profession'].value_counts().sort_index()

Student                           138
Intern                             14
Self-employed                      25
Employed (fixed-term contract)     41
Employed (permanent contract)      70
Unemployed                         20
Other                              21
Name: Profession, dtype: int64

In [36]:
x = df['Profession'].value_counts(normalize=True).sort_index(ascending=False) * 100
ax = x.plot.barh(**pcfg.bar_kwargs, color=pcfg.random_color())
ax.set_xlabel('%')
pcfg.hide_spines(ax)
pcfg.save_fig('dem04_profession')

## Response analysis

In [37]:
from matplotlib_venn import venn3

### Customers

#### Total

In [38]:
# Will be useful later
avg_percent_of_yes = df['Customer'].value_counts(normalize=True).loc['Yes'] * 100

In [39]:
df['Customer'].value_counts().plot.pie(**pcfg.pie_kwargs)
pcfg.save_fig('res01_customer_00')

#### By gender

In [40]:
by_gender = pd.crosstab(df['Customer'], df['Gender'], normalize='columns')
display(by_gender)

Gender,Male,Female,Other
Customer,,,
Yes,0.0,0.452055,0.333333
No,1.0,0.547945,0.666667


In [41]:
kwargs = pcfg.pie_kwargs
kwargs.update({
    'legend':       False,
    'subplots':     True,
    'layout':       (1,3),
    'title':        list(by_gender.columns),
})
by_gender.plot.pie(**kwargs)
pcfg.save_fig('res01_customer_01_gender')

#### By age

In [42]:
by_age = pd.crosstab(df['Customer'], df['AgeGroup'], normalize='columns')
display(by_age)

AgeGroup,<25,25-35,35-45,>45
Customer,,,,
Yes,0.322581,0.456693,0.5,0.588235
No,0.677419,0.543307,0.5,0.411765


In [43]:
y = by_age.iloc[0] * 100
ax = y.plot.bar(**pcfg.bar_kwargs, color=pcfg.random_color())
ax.axhline(avg_percent_of_yes, **pcfg.hline_kwargs)
ax.set_xlabel('Age')
ax.set_ylabel('Customers (%)')
pcfg.hide_spines(ax)
pcfg.save_fig('res01_customer_02_age')

#### By education

In [44]:
by_education = pd.crosstab(df['Customer'], df['Education'], normalize='columns')
display(by_education)

Education,Middle school,High school,Bachelor's degree,Master's degree,Postgraduate specialisation,PhD
Customer,,,,,,
Yes,0.416667,0.356522,0.392523,0.43662,0.55,0.75
No,0.583333,0.643478,0.607477,0.56338,0.45,0.25


In [45]:
kwargs = pcfg.bar_kwargs
kwargs['rot'] = 60
y = by_education.iloc[0] * 100
ax = y.plot.bar(**kwargs, color=pcfg.random_color())
ax.axhline(avg_percent_of_yes, **pcfg.hline_kwargs)
ax.set_xlabel('')
ax.set_ylabel('Customers (%)')
pcfg.hide_spines(ax)
pcfg.save_fig('res01_customer_03_education')

#### By profession

In [46]:
by_profession = pd.crosstab(df['Customer'], df['Profession'], normalize='columns')
display(by_profession)

Profession,Student,Intern,Self-employed,Employed (fixed-term contract),Employed (permanent contract),Unemployed,Other
Customer,,,,,,,
Yes,0.297101,0.285714,0.52,0.414634,0.571429,0.45,0.428571
No,0.702899,0.714286,0.48,0.585366,0.428571,0.55,0.571429


In [47]:
kwargs = pcfg.bar_kwargs
kwargs['rot'] = 60
y = by_profession.iloc[0] * 100
ax = y.plot.bar(**kwargs, color=pcfg.random_color())
ax.axhline(avg_percent_of_yes, **pcfg.hline_kwargs)
ax.set_xlabel('')
ax.set_ylabel('Customers (%)')
pcfg.hide_spines(ax)
pcfg.save_fig('res01_customer_04_profession')

### Purchase frequency

#### By age

In [48]:
by_age = pd.crosstab(df['Frequency'], df['AgeGroup'], normalize='columns', dropna=False)
pcfg.heatmap(by_age, 'res02_frequency_01_age', xlabel='Age', ylabel='Purchase frequency', origin='lower')

AgeGroup,<25,25-35,35-45,>45
Frequency,,,,
1-2 times a year,0.84,0.741379,0.8,0.4
3-4 times a year,0.06,0.206897,0.2,0.5
5-6 times a year,0.10,0.051724,0.0,0.0
7-8 times a year,0.00,0.000000,0.0,0.1
> 9 times a year,0.00,0.000000,0.0,0.0


4 5
